In [1]:
import torch.nn.functional as F
import torch.nn as nn
import torch

from tqdm import tqdm, trange

import os

N = 2000
batch_size = 32

vocab_size = 8000
embedding_dim = 10

seq_len, hidden_dim = 1150, 1200        # change this parameter to test

In [2]:
class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        return self.fc(x).squeeze(-1)

In [3]:
sum([i.numel() for i in TestModel().parameters()])

4444401

In [4]:
x = torch.randint(0, vocab_size, [N, seq_len]).long()
y = torch.randint(0, 1, [N, seq_len]).float()

In [5]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# torch.backends.cudnn.enabled = False

model = TestModel().cuda()

for i in trange(0, N, batch_size):
    x_batch = x[i:i+batch_size].cuda()
    y_batch = y[i:i+batch_size].cuda()

    y_pred = model(x_batch)

    with torch.no_grad():
        F.binary_cross_entropy_with_logits(y_pred.detach(), y_batch)
        pass

 14%|█▍        | 9/63 [00:01<00:11,  4.78it/s]


RuntimeError: CUDA error: unspecified launch failure